In [1]:
#Imorting dependencies
import pandas as pd
from sqlalchemy import inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from postgres_settings import postgresql as settings
from flask import Flask, jsonify

In [2]:
#Reading the Mines Dataset
mines= "Resources/clean_mines.csv"
mines_df = pd.read_csv( mines)
mines_df.head()

,site_code,short_title,stage,lon,lat,target_group_name
0,S0001726,Boddington Bauxite,Operating,116.455902,-32.931561,BAUXITE - ALUMINA
1,S0001739,Willowdale,Operating,116.014801,-32.920719,BAUXITE - ALUMINA
2,S0003638,Marradong,Operating,116.434898,-32.836769,BAUXITE - ALUMINA
3,S0003656,O'Neil,Operating,116.201500,-32.515442,BAUXITE - ALUMINA
4,S0005694,Myara S,Operating,116.059470,-32.498088,BAUXITE - ALUMINA


In [3]:
protocol = 'postgresql'
username = settings['user']
password = settings['passwrd']
host = settings['host']
port = settings['port']
database_name = settings['db']
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

In [4]:
insp.get_table_names()

['mines_db', 'clean_mines_db']

In [23]:
#Reading the Commodities Price Dataset
commodities= "https://raw.githubusercontent.com/ParkaviMathi/Project_3/main/resources/commodity_price.csv"
commodities_price_df = pd.read_csv(commodities)
commodities_price_df.head()

,Commodity,Price,% Change,+/-,Unit,Date
0,Gold,"1,645.28",-0.27%,-4.42,USD per Troy Ounce,10/24/2022
1,Aluminium,"2,181.25",-1.53%,-33.96,USD per Ton,10/24/2022
2,Iron Ore,94.51,-0.37%,-0.35,per Dry Metric Ton,10/24/2022
3,Nickel,"22,150.00",2.80%,602.50,USD per Ton,10/24/2022


In [24]:
postgresql = { 'user' : 'postgres',
                'passwrd' : 'postgres',
                'host' : 'localhost',
                'port' : 5432,
                'db' : 'Mines_Project3'
}

In [32]:
import psycopg2

conn = psycopg2.connect(host=postgresql["host"],
                        port=postgresql["port"],
                        user=postgresql["user"],
                        password=postgresql["passwrd"],
                        database=postgresql["db"]) # To remove slash

cursor = conn.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS commodities_price(
Commodity VARCHAR,
Price INT,
Percentage_change INT,
Variation INT,
Unit VARCHAR,
Last_Updated DATE
);
"""

cursor.execute(create_table_query)

conn.commit()

for index, row in commodities_price_df.iterrows():
    comodity = row["Commodity"]
    price = float(str(row['Price']).replace(",",""))
    change_percent = float(str(row['% Change']).replace(",","").replace("%",""))
    variation = float(str(row["+/-"]).replace(",",""))
    unit = row["Unit"]
    date_arr = row["Date"].strip().split("/")
    date = "-".join([date_arr[2],date_arr[0],date_arr[1]])
    
    cursor.execute("INSERT INTO commodities_price (Commodity, Price, Percentage_change, Variation, Unit, Last_Updated) VALUES(%s, %s, %s, %s, %s, %s)", (comodity,price,change_percent,variation,unit,date))
    
conn.commit() # <- We MUST commit to reflect the inserted data
cursor.close()
conn.close()

In [35]:

conn = psycopg2.connect(host=postgresql["host"],
                        port=postgresql["port"],
                        user=postgresql["user"],
                        password=postgresql["passwrd"],
                        database=postgresql["db"]) # To remove slash

cursor = conn.cursor()

select_query = """SELECT * from commodities_price"""
cursor.execute(select_query)
records = cursor.fetchall()
print("Total rows are:  ", len(records))
print("Printing each row")
for row in records:
    print("Commodity: ", row[0])
    print("Price: ", row[1])
    print("Percentage_change: ", row[2])
    print("Variation: ", row[3])
    print("Unit: ", row[4])
    print("Last_Updated: ", row[5])
    print("\n")
cursor.close()
conn.close()

Total rows are:   4
Printing each row
Commodity:  Gold
Price:  1645
Percentage_change:  0
Variation:  -4
Unit:  USD per Troy Ounce
Last_Updated:  2022-10-24


Commodity:  Aluminium
Price:  2181
Percentage_change:  -2
Variation:  -34
Unit:  USD per Ton
Last_Updated:  2022-10-24


Commodity:  Iron Ore
Price:  95
Percentage_change:  0
Variation:  0
Unit:  per Dry Metric Ton
Last_Updated:  2022-10-24


Commodity:  Nickel
Price:  22150
Percentage_change:  3
Variation:  603
Unit:  USD per Ton
Last_Updated:  2022-10-24


